In [13]:
import gurobipy as gb
import numpy as np
import matplotlib.pyplot as plt

In [14]:
T = 20; n = 4
T = list(range(1,T+1)); G = list(range(n))

################ Adulyasak et al. 2014
# C = floor(2*sum(d)/len(T))
################ Archetti et al. 2011
# h0 = 8; f = 100*10*8; d = U[5, 25]

h = 8; c = 5*h; f = 2*15*h

# Results functions

In [15]:
def print_transportation_summary(mm, setup, prod, hold, T, G, Tt, f, c, h, C, d, y, x, r, l, w, ntabs=0):

    print("\n"+"\t"*ntabs+f"Objective: {mm.objVal:.1f}")
    print("\t"*ntabs+f"\tSetup cost: {setup.getValue():.1f}")
    print("\t"*ntabs+f"\tProduction cost: {prod.getValue():.1f}")
    print("\t"*ntabs+f"\tHolding cost: {hold.getValue():.1f}\n")

    print("\t"*ntabs+" ", *[" "*(5-len(str(t)))+str(t) for t in T], "Total", sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))
    print("\t"*ntabs+"d", *[" "*(5-len(f"{d[t]:.1f}"))+f"{d[t]:.1f}" for t in T], " "*(5-len(f"{sum(d[t] for t in T):.1f}"))+f"{sum(d[t] for t in T):.1f}", sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))
    print("\t"*ntabs+"q", *[" "*(5-len(f"{sum(x[t,m] for m in Tt[t]):.1f}"))+f"{sum(x[t,m] for m in Tt[t]):.1f}" if sum(x[t,m] for m in Tt[t]) > 0 else "    ." for t in T], " "*(5-len(f"{sum(x[t,m] for t in T for m in Tt[t]):.1f}"))+f"{sum(x[t,m] for t in T for m in Tt[t]):.1f}", sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))

    for g in G[:-1]:
        print("\t"*ntabs+f"r{g}", *[(" "*(5-len(f"{r[t,g]:.1f}"))+f"{r[t,g]:.1f}" if round(r[t,g],1) > 0 else "    .") if t in range(1, n-g-1 + 1) else "     " for t in T], sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))

    for t in T:
        print("\t"*ntabs+f"x{t}", *[(" "*(5-len(f"{x[t,m]:.1f}"))+f"{x[t,m]:.1f}" if x[t,m] > 0 else "    .") if m in Tt[t] else "     " for m in T], sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))

    print("\t"*ntabs+"l", *[" "*(5-len(f"{l[t]:.1f}"))+f"{l[t]:.1f}" if round(l[t],1) > 0 else "    ." for t in T], " "*(5-len(f"{sum(l[t] for t in T):.1f}"))+f"{sum(l[t] for t in T):.1f}", sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))
    print("\t"*ntabs+"w", *[" "*(5-len(f"{w[t]:.1f}"))+f"{w[t]:.1f}" if round(w[t],1) > 0 else "    ." for t in T], " "*(5-len(f"{sum(w[t] for t in T):.1f}"))+f"{sum(w[t] for t in T):.1f}", sep="\t")


def print_summary(m, T, G, f, c, h, C, d, y, x, z, l, w, I, ntabs = 0):

    print("\n"+"\t"*ntabs+f"Objective: {round(m.getObjective().getValue(),2)}\tRuntime: {round(m.RunTime,5)}. C = {round(C,1)}")
    print("\t"*ntabs+f"\tSetup cost: {round(sum(f*y[t] for t in T),2)}")
    print("\t"*ntabs+f"\tProduction cost: {round(sum(c*x[t] for t in T),2)}")
    print("\t"*ntabs+f"\tHolding cost: {round(sum(h*I[t,g] for t in T for g in G),2)}\n")

    print("\t"*ntabs+" ", *[" "*(5-len(str(t)))+str(t) for t in T], "Total", sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))
    print("\t"*ntabs+"d", *[" "*(5-len(str(round(d[t],1))))+str(round(d[t],1)) for t in T], " "*(5-len(str(round(sum(d[t] for t in T),1))))+str(round(sum(d[t] for t in T),1)), sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))
    print("\t"*ntabs+"x", *[" "*(5-len(str(round(x[t],1))))+str(round(x[t],1)) if round(x[t],1) > 0 else "    ." for t in T], " "*(5-len(str(round(sum(x[t] for t in T),1))))+str(round(sum(x[t] for t in T),1)), sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))

    for g in G:
        print("\t"*ntabs+f"z{g}", *[" "*(5-len(str(round(z[t,g],1))))+str(round(z[t,g],1)) if round(z[t,g],1) > 0 else "    ." for t in T],  " "*(5-len(str(round(sum(z[t,g] for t in T),1))))+str(round(sum(z[t,g] for t in T),1)), sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))
    
    print("\t"*ntabs+"l", *[" "*(5-len(str(round(l[t],1))))+str(round(l[t],1)) if round(l[t],1) > 0 else "    ." for t in T], " "*(5-len(str(round(sum(l[t] for t in T),1))))+str(round(sum(l[t] for t in T),1)), sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))

    for g in G[:-1]:
        print("\t"*ntabs+f"I{g}", *[" "*(5-len(str(round(I[t,g],1))))+str(round(I[t,g],1)) if round(I[t,g],1) > 0 else "    ." for t in T],  " "*(5-len(str(round(sum(I[t,g] for t in T),1))))+str(round(sum(I[t,g] for t in T),1)), sep="\t")
    print("\t"*ntabs+"-"*8*(3+len(T)))

    print("\t"*ntabs+"w", *[" "*(5-len(str(round(w[t],1))))+str(round(w[t],1)) if round(w[t],1) > 0 else "    ." for t in T], " "*(5-len(str(round(sum(w[t] for t in T),1))))+str(round(sum(w[t] for t in T),1)), sep="\t")

    print("\n"+"\t"*ntabs+f"Fresh fill rate: {sum(z[t,0] for t in T)/sum(d[t] for t in T):.1%}, Total fill rate: {sum(z[t,g] for t in T for g in G)/sum(d[t] for t in T):.1%}")


def check_models_objectives(it, T, G, f, c, h, y, x, I, ntabs = 0, s = "Transportation"):

    objs = [round(sum(f*y[m][t] + c*x[m][t] + h*sum(I[m][t,g] for g in G) for t in T),2) for m in [0,1]]; resp = True
    if objs[0] != objs[1] and np.abs(objs[0]-objs[1])/np.min(objs) > 1e-4:
        print("\t"*ntabs+f"{it} DIFFERENT OBJECTIVE VALUES - Standard: {objs[0]}, {s}: {objs[1]}"); resp = False

    return resp


# Models functions

In [16]:
def standard_model(T, G, n, h, c, f, C, I0, d, alpha, beta = 0, epsilon = 1, output = False, verbose = False, FIFO = 0, feasibility = False):

    m = gb.Model("Standard Model")
    Tt = {t:[m for m in range(t,np.min((T[-1],t+n-1)) + 1)] for t in T}

    ######################################
    # DECISION VARIABLES
    ######################################

    # Whether there is production in period t \in T or not
    y = {t:m.addVar(name=f"y_{t}", vtype=gb.GRB.BINARY) for t in T}
    # Produced quantity in period t \in T
    x = {t:m.addVar(name=f"x_{t}", vtype=gb.GRB.CONTINUOUS) for t in T}
    # Amount of product of age g \in G that is used to fulfill demand in period t \in T
    z = {(t,g):m.addVar(name=f"z_{t,g}", vtype=gb.GRB.CONTINUOUS) for t in T for g in G}
    # Available inventory of product of age g \in G at the end of period t \in T
    I = {(t,g):m.addVar(name=f"I_{t,g}", vtype=gb.GRB.CONTINUOUS) for t in T for g in G}
    I.update({(0,g):I0[g] for g in I0})

    ######################################
    # CONSTRAINTS
    ######################################

    for t in T:
        # Inventory of fresh produce
        m.addConstr(I[t,0] == x[t] - z[t,0])

        for g in G[1:]: 
            # Inventory dynamics throughout the day   
            m.addConstr(I[t,g] == I[t-1,g-1] - z[t,g])
        
        # Production capacity
        m.addConstr(x[t] <= np.min((C,sum(d[m] for m in Tt[t])))*y[t])

        # Demand fulfillment and lost sales modeling
        m.addConstr(gb.quicksum(z[t,g] for g in G) <= d[t])

        # Individual demand service level constraint
        m.addConstr(gb.quicksum(z[t,k] for k in G)/d[t] >= beta)

    for g in G:
        # Total age service level constraint
        m.addConstr(gb.quicksum(z[t,k] for t in T for k in range(g+1)) >= alpha[g]*sum(d[t] for t in T))
    
    # Total waste control constraint
    m.addConstr(gb.quicksum(I[t,n-1] for t in T) <= epsilon*gb.quicksum(x[t] for t in T))

    if FIFO == 1:
        gamma = {(t,g):m.addVar(name=f"gamma_{t,g}", vtype=gb.GRB.BINARY) for t in T for g in G}

        for t in T:

            for g in G:
                m.addConstr(I[t,g] <= C*(1-gamma[t,g]))

                if g < G[-1]:
                    m.addConstr(gamma[t,g] <= gamma[t,g+1])
                    m.addConstr(z[t,g] <= C*gamma[t,g+1])
    
    ######################################
    # OBJECTIVE FUNCTION
    ######################################
            
    m.setObjective(gb.quicksum(f*y[t] + c*x[t] + h*gb.quicksum(I[t,g] for g in G) for t in T))

    m.update()
    m.setParam("OutputFlag",output)
    m.optimize()

    ######################################
    # RESULTS RETRIEVING
    ######################################

    y = {t:round(y[t].x) for t in T}
    x = {t:x[t].x for t in T}
    z = {(t,g):z[t,g].x for t in T for g in G}
    l = {t:d[t]-sum(z[t,g] for g in G) for t in T}
    w = {t:I[t,n-1].x for t in T}
    I = {(t,g):I[t,g].x for t in T for g in G}

    if feasibility:
        xx = {(t,m):z[m,m-t] for t in T for m in Tt[t]}
        r = {(t,g):z[t,g+t] for g in G[:-1] for t in range(1, n-g-1 + 1)}

        for t in T:
            if t <= n-1 and round(sum(r[t,g] for g in range(n-t-1 + 1)) + sum(xx[j,t] for j in range(1,t+1)) + l[t],3) != round(d[t],3): print(f"\t\tDemand modeling for period {t} doesn't add up: {round(sum(r[t,g] for g in range(n-t-1 + 1)) + sum(xx[j,t] for j in range(1,t+1)) + l[t],3)} != {round(d[t],3)}")
            if t >= n and round(sum(xx[j,t] for j in range(t-n+1,t+1)) + l[t],3) != round(d[t],3): print(f"\t\tDemand modeling for period {t} doesn't add up: {round(sum(xx[j,t] for j in range(1,t+1)) + l[t],3)} != {round(d[t],3)}")

            for m in Tt[t]:
                if round(xx[t,m],3) > round(np.min((C,d[m]))*y[t],3): print(f"\t\tProduction capacity for period {t} is violated: {round(xx[t,m],3)} > {round(np.min((C,d[m]))*y[t],3)}")
            if t+n-1 <= T[-1] and round(sum(xx[t,m] for m in Tt[t]) + w[t+n-1],3) > round(C*y[t],3): print(f"\t\tTotal production capacity for period {t} is violated: {round(sum(xx[t,m] for m in Tt[t]) + w[t+n-1],3)} > {round(C*y[t],3)}")
            if t+n-1 > T[-1] and round(sum(xx[t,m] for m in Tt[t]),3) > round(C*y[t],3): print(f"\t\tTotal production capacity for period {t} is violated: {round(sum(xx[t,m] for m in Tt[t]),3)} > {round(C*y[t],3)}")

        for g in G[:-1]:
            if round(sum(r[t,g] for t in range(1, n-g-1 + 1)) + w[n-g-1],3) != round(I0[g],3): print(f"\t\tInitial inventory of age {g} doesn't add up: {round(sum(r[t,g] for t in range(1, n-g-1 + 1)) + w[n-g-1],3)} != {round(I0[g],3)}")

        if round(sum(xx[t,t] for t in T)/sum(d[t] for t in T),3) < alpha[0]: print(f"\t\tFresh produce fill rate violated: {round(sum(xx[t,t] for t in T)/sum(d[t] for t in T),3)} < {alpha[0]}")
        if round((sum(xx[t,m] for t in T for m in Tt[t] if m-t<=G[-1]) + sum(r[t,k] for t in range(1, G[-1]+1) for k in range(G[-1]-t+1)))/sum(d[t] for t in T),3) < alpha[G[-1]]: print(f"\t\tTotal fill rate violated: {round((sum(xx[t,m] for t in T for m in Tt[t] if m-t<=G[-1]) + sum(r[t,k] for t in range(1, G[-1]+1) for k in range(G[-1]-t+1)))/sum(d[t] for t in T),3)} < {alpha[G[-1]]}")

    if verbose: print_summary(m, T, G, f, c, h, C, d, y, x, z, l, w, I, ntabs=2)

    return y, x, z, l, w, I, round(m.objVal,3)
    
def transportation_model(T, G, n, h, c, f, C, I0, d, alpha, beta = 0, epsilon = 1, output = False, verbose = False, feasibility = False):

    mm = gb.Model("Standard Model")
    Tt = {t:[m for m in range(t,np.min((T[-1],t+n-1)) + 1)] for t in T}

    cw = {t:0 if t<=n-1 else c for t in T}
    hx = {(t,m):h*(m-t) for t in T for m in Tt[t]}
    hw = {t:h*t if t <= n-1 else h*n for t in T}
    hr = {t:h*(t-1) for t in T}

    ######################################
    # DECISION VARIABLES
    ######################################

    # Whether there is production in time period t \in T or not
    y = {t:mm.addVar(name=f"y_{t}",vtype=gb.GRB.BINARY) for t in T}
    # Production quantity of time period t \in \T
    x = {(t,m):mm.addVar(name=f"x_{t,m}",vtype=gb.GRB.CONTINUOUS) for t in T for m in Tt[t]}
    # Amount of initial inventory of age g \in G - {n-1} used to fulfill demand in time period t \in {1, ..., n-g-1}
    r = {(t,g):mm.addVar(name=f"r_{t,g}",vtype=gb.GRB.CONTINUOUS) for g in G[:-1] for t in range(1,n-g)}
    # Generated waste in time period t \in T
    w = {t:mm.addVar(name=f"w_{t}",vtype=gb.GRB.CONTINUOUS) for t in T}

    ######################################
    # CONSTRAINTS
    ######################################      
    
    for g in G[:-1]:
        # Initial inventory modeling
        mm.addConstr(I0[g] == gb.quicksum(r[t,g] for t in range(1, n-g-1 + 1)) + w[n-g-1])
    
    for t in T:
        # Demand modeling
        if t <= n-1:
            mm.addConstr(gb.quicksum(r[t,g] for g in range(n-t-1 + 1)) + gb.quicksum(x[j,t] for j in range(1, t+1)) <= d[t])
        else:
            mm.addConstr(gb.quicksum(x[j,t] for j in range(t-n+1, t+1)) <= d[t])
        
        # Production capacity constraints
        if t+n-1 <= T[-1]:
            mm.addConstr(gb.quicksum(x[t,m] for m in Tt[t]) + w[t+n-1] <= C*y[t])
        else:
            mm.addConstr(gb.quicksum(x[t,m] for m in Tt[t]) <= C*y[t])
        for m in Tt[t]:
            mm.addConstr(x[t,m] <= np.min((C,d[m]))*y[t])

    # Age service level constraints
    mm.addConstr(gb.quicksum(x[t,t] for t in T) >= alpha[0]*sum(d[t] for t in T))
    mm.addConstr(gb.quicksum(x[t,m] for (t,m) in x) + gb.quicksum(r[t,g] for (t,g) in r) >= alpha[n-1]*sum(d[t] for t in T))
    
    # Demand service level per time period
    for t in T:
        if t <= n-1:
            mm.addConstr(gb.quicksum(r[t,g] for g in range(n-t-1 + 1)) + gb.quicksum(x[j,t] for j in range(1, t+1)) >= beta*d[t])
        else:
            mm.addConstr(gb.quicksum(x[j,t] for j in range(t-n+1, t+1)) >= beta*d[t])

    # Waste control constraint
    mm.addConstr(gb.quicksum(w[t] for t in T) <= epsilon*(gb.quicksum(x[t,m] for t in T for m in Tt[t])+gb.quicksum(w[t] for t in T if t >= n)))
    
    ######################################
    # OBJECTIVE FUNCTION
    ######################################
    
    setup_cost = gb.quicksum(f*y[t] for t in T)
    production_cost = gb.quicksum(c*gb.quicksum(x[t,m] for m in Tt[t]) for t in T) + gb.quicksum(c*w[t] for t in T if t >= n)
    holding_cost = gb.quicksum(h*(m-t)*x[t,m] for t in T for m in Tt[t]) + gb.quicksum(h*(t-1)*r[t,g] for (t,g) in r) + gb.quicksum(h*t*w[t] if t <= n-1 else h*n*w[t] for t in T)
    mm.setObjective(gb.quicksum(f*y[t] + gb.quicksum((c+hx[t,m])*x[t,m] for m in Tt[t]) + (cw[t]+hw[t])*w[t] + gb.quicksum(hr[t]*r[t,g-t] for g in G if t-g<=0) for t in T))

    mm.update()
    mm.setParam("OutputFlag",output)
    mm.optimize()

    ######################################
    # RESULTS RETRIEVING
    ######################################

    y = {t:round(y[t].x) for t in T}
    xx = {t:sum(x[t,m].x for m in Tt[t])+w[t+n-1].x if t+n-1 <= T[-1] else sum(x[t,m].x for m in Tt[t]) for t in T}
    z = {(t,g):x[t-g,t].x if t-g >= 1 else r[t,g-t].x for t in T for g in G}
    l = {t:d[t]-sum(z[t,g] for g in G) for t in T}
    w = {t:w[t].x for t in T}
    I = {(0,g):I0[g] for g in I0}
    for t in T:
        for g in G[:-1]:

            if t-g<=0: I[t,g] = sum(r[j,g-t].x for j in range(t+1, (n-1)-(g-t) + 1)) + w[(n-1)-(g-t)]
            elif t-g <= T[-1]-n+1: I[t,g] = sum(x[t-g, m].x for m in Tt[t-g] if m > t) + w[(n-1)+(t-g)]
            elif t < T[-1]: I[t,g] = sum(x[t-g, m].x for m in Tt[t-g] if m > t)
        
        I[t,n-1] = w[t]

    for g in G[:-1]:
        I[T[-1],g] = 0

    x = {(t,m):x[t,m].x for (t,m) in x}
    r = {(t,g):r[t,g].x for (t,g) in r}

    ######################################
    # FEASIBILITY CHECK
    ######################################
    if feasibility:
        for t in T:
            if round(I[t,0],3) != round(xx[t]-z[t,0],3): print(f"\t\tFresh produce inventory of period {t} doesn't add up. {round(I[t,0],3)} != {round(xx[t]-z[t,0],3)}")
            for g in G[1:]:
                if round(I[t,g],3) != round(I[t-1,g-1]-z[t,g],3): print(f"\t\tInventory of age {g} in period {t} doesn't add up. {I[t,g],3} - {round(I[t-1,g-1]-z[t,g],3)}")

            if round(xx[t],3) > round(np.min((C,sum(d[m] for m in Tt[t])))*y[t],3): print(f"\t\tProduction capacity constraint for day {t} is violated. {xx[t]} > {C*y[t]}")
            dem = round(sum(z[t,g] for g in G) + l[t],3)
            if dem != round(d[t],3): print(f"\t\tDemand modeling of period {t} doesn't add up. {dem} != {d[t]}")

        for g in G:
            sl = round(sum(z[t,k] for t in T for k in range(0, g+1))/sum(d[t] for t in T),3)
            if sl < alpha[g]: print(f"\t\tTotal age service level constraint for age {g} is violated. {sl} < {alpha[g]}")
        
        waste = sum(I[t, n-1] for t in T)/sum(xx[t] for t in T)
        if waste > epsilon: print(f"\t\tTotal waste level constraint is violated. {waste} > {epsilon}")

    if verbose:
        print_transportation_summary(mm, setup_cost, production_cost, holding_cost, T, G, Tt, f, c, h, C, d, y, x, r, l, w, ntabs=2)
        print_summary(mm, T, G, f, c, h, C, d, y, xx, z, l, w, I, ntabs=2)

    return y, xx, z, l, w, I, round(mm.objVal,3)

# Other

In [32]:
verbose = False; reps = 1; beta = 0
I0 = {g:0 for g in G[:-1]}

for sl in range(7,11):

    print(f"\nChecking for total service level of {sl/10:.0%}")
    alpha = {g:0 if g < G[-1] else sl/10 for g in G}

    for aa in range(reps):
        d = {t:np.random.rand()*20+5 for t in T}; C = np.floor(2*sum(list(d.values()))/len(T))
        
        y1, x1, z1, l1, w1, I1, obj1 = standard_model(T, G, n, h, c, f, C, I0, d, alpha, beta=beta, verbose=verbose)
        y2, x2, z2, l2, w2, I2, obj2 = transportation_model(T, G, n, h, c, f, C, I0, d, alpha, beta=beta)
        
        if np.abs(obj1-obj2)/np.min([obj1,obj2]) > 1e-4:
            print("\t"*2+f"DIFFERENT OBJECTIVE VALUES - Standard: {obj1}, Transportation: {obj2}")




Checking for total service level of 70%

Checking for total service level of 80%

Checking for total service level of 90%

Checking for total service level of 100%


# Formulations equivalence check

In [ ]:

for tt in [10,2030]:
    for n in [4,8]:
        
        if n <= tt/2:
            
            T = list(range(1,tt+1)); G = list(range(n))
            I0 = {g:10 for g in G[:-1]}
            print(f"Checking for |T| = {tt}, n = {n}")

            for a0 in [0.4, 0.6]:
                for a1 in [0.8, 1]:

                    alpha = {g:a0 if g < G[-1] else a1 for g in G}
                    print(f"\talpha_0 = {a0}, alpha_n1 = {a1}")

                    correct = 0
                    while correct <= 5e3:
                        
                        #try:
                        d = {t:np.random.rand()*20+5 for t in T}
                        C = np.floor(2*sum(list(d.values()))/len(T))

                        y1, x1, z1, l1, w1, I1, obj1 = standard_model(T, G, n, h, c, f, C, I0, d, alpha, FIFO = False, verbose=False)
                        y2, x2, z2, l2, w2, I2, obj2 = transportation_model(T, G, n, h, c, f, C, I0, d, alpha, verbose=False)
                        #resp = check_models_objectives(correct, T, G, f, c, h, (y1,y2), (x1,x2), (I1,I2), ntabs=2)

                        if np.abs(obj1-obj2)/np.min([obj1,obj2]) > 1e-4:
                            if obj2 < obj1: message = "TF is lower"
                            else: message = "ST is lower"
                            print("\t"*2+f"{correct} DIFFERENT OBJECTIVE VALUES - Standard: {obj1}, Transportation: {obj2}. {message}\n")

                            y1, x1, z1, l1, w1, I1, obj1 = standard_model(T, G, n, h, c, f, C, I0, d, alpha, FIFO = False, verbose=True)
                            y2, x2, z2, l2, w2, I2, obj2 = transportation_model(T, G, n, h, c, f, C, I0, d, alpha, verbose=True)
                        #if not resp:
                        #    y1, x1, z1, l1, w1, I1 = standard_model(T, G, n, h, c, f, C, I0, d, alpha, feasibility = True)
                        #    y2, x2, z2, l2, w2, I2 = transportation_model(T, G, n, h, c, f, C, I0, d, alpha, feasibility = True)#

                        correct += 1
                        if correct % 200 == 0: print(f"\t\t --- DONE {correct}")

                        #except:
                        #    continue